In [1]:
import torch
from torch import Tensor
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader

from dataset import QM7


device = 'cuda'

ds_train = QM7(root='qm7', fold=0, train=True)
ds_test = QM7(root='qm7', fold=0, train=False)

loader = DataLoader(ds_train, batch_size=64, shuffle=True)

c:\Users\20tod\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing...
Done!
Processing...
Done!


In [3]:
from tqdm import tqdm

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(-1, 128)
        self.conv2 = GCNConv(128, 256)
        self.fc = Linear(256, 1)


    def forward(self, x: Tensor, edge_index: Tensor, edge_weight: Tensor, batch) -> Tensor:
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = self.conv2(x, edge_index, edge_weight).relu()
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x

model = GCN().to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


epochs = 100
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(loader)*epochs)

for epoch in tqdm(range(epochs)):
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        loss = criterion(out, batch.y.unsqueeze(1))
        loss.backward()
        optimizer.step()
        # scheduler.step()

100%|██████████| 100/100 [03:26<00:00,  2.06s/it]


In [4]:
test_loader = DataLoader(ds_train, batch_size=128, shuffle=True)
model.eval()
pred = []
gt = []
ae = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        pred.append(out.detach().cpu())
        gt.append(batch.y.unsqueeze(1).cpu())
        ae.append((out.detach().cpu() - batch.y.unsqueeze(1).cpu()).numpy())

In [5]:
pred[0][6], gt[0][6]

(tensor([0.8499]), tensor([1.0512]))

In [6]:
len(ae)

56

In [7]:
import numpy as np
ae = np.concatenate(ae, axis=0)

In [8]:
np.abs(ae).mean()*223.91891

49.63631204289854